<h1 style="text-align: center;">Tennis Analyzer ML Model V2</h1>
<h3 style="text-align: center;">Dan Warnick</h3>

<p>To start we will begin by selecting the data points we want to analyze with existing known results. For each data entry we will have two players each with the following data entries.</p>
<table style="font-size: .8em;">
    <tr>
        <th>Player Name</th>
    </tr>
</table>
<p>Along with match facts like Clay/Hard/Grass Court or Indoor/Outdoor. In the future may want to add weather and adjust certain parameters for more accuracy and more data points to train from, however for now this seems a good start.</p>

<h2>1.) Collect Data Efficiently</h2>

In [331]:
import django
from django.http import HttpResponse
from django.template import loader
import os
from django.db import models
import torch
from math import ceil, floor
import math
import numpy as np
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils as utils
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from asgiref.sync import sync_to_async # type: ignore
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
import copy
import joblib
import lightgbm as lgb
import shap
from optuna.integration import lightgbm as lgb_optuna

from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'breakpoint.settings')
# django.setup()

# from render.models import *

In [332]:
start = '20120101'
end = '20231231'
match_type = 'm'

In [333]:
start_date = datetime.strptime(start, '%Y%m%d').date()
end_date = datetime.strptime(end, '%Y%m%d').date()

# if match_type == 'm':   
#     typer = MensTennisMatch
#     insert_db = MensTennisMatchStats
# else:
#     typer = WomensTennisMatch
#     insert_db = WomensTennisMatchStats

# query = insert_db.objects.filter(
#         tourney_date__range=(start_date, end_date)
#     ).order_by('tourney_date')
    
# games = await sync_to_async(list)(query.all().values())

df = pd.read_csv('testcsvs/testout.csv')
df['tourney_date'] = pd.to_datetime(df['tourney_date']).dt.date
df = df[(df['tourney_date'] >= start_date) & (df['tourney_date'] <= end_date)]
df = df[(df['a_glicko_rd'] <= 150) & (df['b_glicko_rd'] <= 150)]

df = df.drop(['tourney_id','match_num', 'tourney_name', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug'], axis=1)

# df = pd.DataFrame(games).drop(['tourney_id', 'tourney_name', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug','a_win_percent', 'a_serve_rating', 'a_return_rating', 'a_pressure_rating', 'a_avg_vs_elo', 'a_matches_played', 'b_win_percent', 'b_serve_rating', 'b_return_rating', 'b_pressure_rating', 'b_avg_vs_elo', 'b_matches_played', 'A_Odds', 'b_odds'], axis=1)

one_hot_encoder = OneHotEncoder(sparse_output=False)

# One-Hot Encode the 'category_text' column
category_encoded = one_hot_encoder.fit_transform(df[['surface']])

# Convert to DataFrame
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names_out(['surface']))

# Concatenate the one-hot encoded columns back to the original DataFrame
df = pd.concat([df, category_encoded_df], axis=1)

# Drop the original 'category_text' column
df.drop('surface', axis=1, inplace=True)
print(len(df))

36152


In [334]:
df.head()

,best_of,a_player_rank,b_player_rank,a_glicko_rating,a_glicko_rd,a_glicko_vol,a_point_glicko_rating,a_point_glicko_rd,a_point_glicko_vol,a_game_glicko_rating,...,b_surface_second_won_glicko_vol,b_surface_return_second_won_glicko_rating,b_surface_return_second_won_glicko_rd,b_surface_return_second_won_glicko_vol,a_b_win,a_odds,b_odds,surface_Clay,surface_Grass,surface_Hard
5373,3.0,15.0,74.0,1839.797067,66.607302,0.059900,1531.323923,62.908167,0.059333,1564.401893,...,0.059353,1514.785695,64.544807,0.059353,1.0,1.28,3.59,0.0,0.0,1.0
5375,3.0,46.0,65.0,1704.915868,66.531031,0.059971,1521.849326,63.670643,0.059581,1543.174689,...,0.058877,1524.305006,62.313940,0.058877,0.0,NaN,NaN,0.0,0.0,1.0
5377,3.0,95.0,89.0,1570.551049,83.226172,0.059989,1511.282842,71.979231,0.059878,1516.780899,...,0.059383,1508.366657,63.335804,0.059383,1.0,1.59,2.29,0.0,0.0,1.0
5378,3.0,48.0,83.0,1731.171432,66.027320,0.059948,1516.808438,63.500630,0.059475,1541.510490,...,0.059427,1507.539322,63.640647,0.059427,0.0,2.40,1.54,0.0,0.0,1.0
5380,3.0,70.0,22.0,1545.639101,79.730461,0.060001,1503.968536,69.668106,0.059875,1503.773430,...,0.058717,1503.340027,62.313771,0.058717,0.0,4.44,1.19,0.0,0.0,1.0


In [335]:
# Create a scatter plot
# df = df[~(df == 1500).any(axis=1)]
# plt.figure(figsize=(5, 5))

# # Plot a_elo_rating vs b_elo_rating for a_b_win == 1
# x1 = df[df['a_b_win'] == 1]['a_recent_elo_rating']
# y1 = df[df['a_b_win'] == 1]['b_recent_elo_rating']
# plt.scatter(x1, y1, color='blue', label='Favorite Wins', s=.5, alpha=0.5)

# # Plot b_elo_rating vs a_elo_rating for a_b_win == 0
# x2 = df[df['a_b_win'] == 0]['b_recent_elo_rating']
# y2 = df[df['a_b_win'] == 0]['a_recent_elo_rating']
# plt.scatter(x2, y2, color='orange', label='Upset', s=.5, alpha=0.5)

# # Combine data for a single trendline
# combined_x = np.concatenate([x1, x2])
# combined_y = np.concatenate([y1, y2])

# # Fit a polynomial of degree 2 to the combined data
# print(combined_x)
# coefficients = np.polyfit(combined_x, combined_y, 3)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(combined_x.min(), combined_x.max(), 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Quadratic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(1100, 2300)
# plt.ylim(1100, 2300)

# # Adding labels and title
# plt.xlabel('ELO Rating Winner')
# plt.ylabel('ELO Rating Loser')
# plt.title('Scatter Plot of ELO Ratings based on Win/Loss')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [336]:
# plt.figure(figsize=(8, 5))

# df['elo_diff'] = df['a_recent_elo_rating']

# # Create bins for ELO difference in intervals of 10
# bins = np.arange(df['elo_diff'].min(), df['elo_diff'].max() + 10, 10)
# labels = (bins[:-1] + bins[1:]) / 2
# df['elo_diff_bin'] = pd.cut(df['elo_diff'], bins=bins, labels=labels, include_lowest=True)

# # Calculate average win rate at each ELO difference bin
# average_win_rate = df.groupby('elo_diff_bin')['a_b_win'].mean().reset_index()
# average_win_rate.columns = ['elo_diff_bin', 'avg_win_rate']

# # Convert the bin labels to numeric values
# average_win_rate['elo_diff_bin'] = average_win_rate['elo_diff_bin'].astype(float)

# average_win_rate = average_win_rate[~np.isnan(average_win_rate).any(axis=1)]

# # Create the scatter plot
# plt.scatter(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], color='blue', label='Average Win Rate', s=10, alpha=0.5)

# # Fit a polynomial of degree 3 to the average win rate data
# coefficients = np.polyfit(average_win_rate['elo_diff_bin'], average_win_rate['avg_win_rate'], 5)
# polynomial = np.poly1d(coefficients)
# trendline_x = np.linspace(-600, 600, 100)
# trendline_y = polynomial(trendline_x)
# plt.plot(trendline_x, trendline_y, color='green', linewidth=1, label='Cubic Trendline')

# # Setting the limits for x and y axis
# plt.xlim(-600, 600)
# plt.ylim(0, 1)

# # Adding labels and title
# plt.xlabel('ELO Rating Difference (Winner - Loser)')
# plt.ylabel('Average Win Rate')
# plt.title('Scatter Plot of ELO Rating Difference vs. Win Rate')
# plt.grid(True)
# plt.legend()

# # Show the plot
# plt.show()

In [337]:
def trainModel(features, df_train, df_validate, df_test):
    y_train = df_train["Result"]
    y_validate = df_validate["Result"]
    y_test = df_test["Result"]

    X_train = df_train[features]
    X_validate = df_validate[features]
    X_test = df_test[features]

    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_eval = lgb.Dataset(X_validate, label=y_validate, reference=lgb_train)

    params = {
        'learning_rate': 0.01,
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boost_from_average': True,
        'feature_pre_filter': False,
    }

    param_grid = {
        'num_leaves': [31, 50, 70],
        'feature_fraction': [0.8, 0.9, 1.0],
        'bagging_fraction': [0.8, 0.9, 1.0],
        'min_child_samples': [20, 30, 40],
        'lambda_l1': [0.0, 0.1, 0.5],
        'lambda_l2': [0.0, 0.1, 0.5],
        'max_depth': [-1, 10, 20],
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # LightGBM model
    gbm = lgb.LGBMClassifier(**params, n_estimators=300)

    # Grid search
    grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, 
                               scoring='roc_auc', cv=skf, verbose=1, n_jobs=-1)

    grid_search.fit(X_train, y_train)

    # Best parameters
    print("Best Parameters: ", grid_search.best_params_)
    
    # Use the best model
    best_model = grid_search.best_estimator_
    
    return best_model

In [338]:
def evaluateModel(model, df_test, features):
    X_test = df_test[features]
    y_test = df_test["Result"]

    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    return {
        'accuracy': accuracy,
        'auc': auc
    }

In [339]:
df = df.dropna()
df = df[~(df == 0.5).any(axis=1)]

# Extract and manage odds columns
odds_df = df[['a_odds', 'b_odds']].copy()
odds_df['index'] = df.index
df = df.drop(columns=['a_odds', 'b_odds'])

# Split the data into features and target
y = df['a_b_win'].values
X = df.drop('a_b_win', axis=1).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, odds_df['index'].values, test_size=0.2, random_state=42
)

# Split the training set further into training and validation sets
X_train, X_validate, y_train, y_validate = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Convert back to DataFrames to match your trainModel function expectations
df_train = pd.DataFrame(X_train, columns=df.columns.drop('a_b_win'))
df_train['Result'] = y_train

df_validate = pd.DataFrame(X_validate, columns=df.columns.drop('a_b_win'))
df_validate['Result'] = y_validate

df_test = pd.DataFrame(X_test, columns=df.columns.drop('a_b_win'))
df_test['Result'] = y_test

# List of features you want to use for training
selected_features = df_train.columns.drop('Result').tolist()  # This includes all features except the target column

# Train the model with hyperparameter tuning and cross-validation
model = trainModel(selected_features, df_train, df_validate, df_test)

# Evaluate the model
evaluation_metrics = evaluateModel(model, df_test, selected_features)

# Print evaluation results
print(f"Accuracy: {evaluation_metrics['accuracy']:.4f}")
print(f"AUC: {evaluation_metrics['auc']:.4f}")

importances = model.feature_importances_

# Create a DataFrame for the feature importances
feature_importance_df = pd.DataFrame({
    'Feature': selected_features,
    'Importance': importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importance
plt.figure(figsize=(15, 15))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()  # To have the most important features on top
plt.show()

# Optionally, save the model to a file
model.booster_.save_model('lightgbm_model.txt')

# Optionally save predictions with odds
y_pred = model.predict(X_test)
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'A_Odds': odds_df.loc[idx_test, 'a_odds'].values,
    'B_Odds': odds_df.loc[idx_test, 'b_odds'].values
})

results_df.to_csv('predictions_with_odds.csv', index=False)

# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
plt.plot(fpr, tpr)
plt.title("Receiver Operating Characteristics")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

Fitting 5 folds for each of 2187 candidates, totalling 10935 fits


In [ ]:
# Display the DataFrame
# results_df.head()
print(results_df)

NameError: name 'results_df' is not defined

In [ ]:
better = 0
total_won = 0
diff_fav = 0
bet_correct = 0

model_correct = 0
vegas_correct = 0
vegas_total = 0

wrong = 0
comparison_df = results_df.dropna()
length = len(comparison_df)

confidence_pct = .7
confidence_top_pct = .8

for i, row in comparison_df.iterrows():

    if confidence_top_pct > row['Predicted'] > confidence_pct and row['Predicted'] > 1/row['A_Odds']:
        better += 1
        if(row['Actual'] == 1):
            bet_correct += 1
            total_won += row['A_Odds']-1
        else:
            wrong += 1
            total_won -= 1

    if 1-confidence_top_pct < row['Predicted'] < 1-confidence_pct and 1-row['Predicted'] > 1/row['B_Odds']:
        better += 1
        if(row['Actual'] == 0):
            bet_correct += 1
            total_won += row['B_Odds']-1
        else:
            wrong += 1
            total_won -= 1

    if round(row['Predicted']) != round(1/row['A_Odds']):
        diff_fav += 1

    if row['Actual']==1 and row['A_Odds'] < row['B_Odds']:
        # print(f"A odds : {row['a_odds']} B odds : {row['b_odds']}")
        vegas_correct += 1

    if row['Actual']==0 and row['A_Odds'] > row['B_Odds']:
        vegas_correct += 1

    if round(row['Predicted']) == round(row['Actual']):
        model_correct += 1


print(f"Total won on $1 bets: {total_won:.2f} on a total # bets: {better} from a total of {length} games")
print(f"Amount of differing favorites {diff_fav/length}")
print(f"Amount of incorrect bets : {wrong/better}")
print(f"Correct Bets: {bet_correct/better}")
print(f"Model % Correct : {model_correct/length} Vegas Correct % : {vegas_correct/length}")
